In [ ]:
import numpy as np
import pandas as pd
import stat
import matplotlib.pyplot as plt
!pip install linearmodels

#Performed pooled OLS
from linearmodels import PooledOLS
import statsmodels.api as sm
from linearmodels import PanelOLS

In [ ]:
path = '/content/Hanselaar_Stulz_vanDijk_data.xlsx'

In [ ]:
Issues = pd.read_excel(path, sheet_name='Issues', skiprows=5)
Issues = Issues.set_index(['YearQuarter']) 
Issues = Issues.diff()
Issues_1 = Issues.shift()
Issues_2 = Issues.shift(2)
Issues_3 = Issues.shift(3)
#Issues['Quarter'] = Issues.YearQuarter % 10 

Liquidity = pd.read_excel(path, sheet_name='Liquidity', skiprows=5)
Liquidity = Liquidity.set_index(['YearQuarter'])  
Liquidity = Liquidity.diff()
Liquidity_2p = Liquidity.shift(-2)
Liquidity_1p = Liquidity.shift(-1)
Liquidity_1 = Liquidity.shift(1)
Liquidity_2 = Liquidity.shift(2)
Liquidity_3 = Liquidity.shift(3)
Liquidity_4 = Liquidity.shift(4)
Liquidity_5 = Liquidity.shift(5)
Liquidity_6 = Liquidity.shift(6)

#Liquidity['Quarter'] = Liquidity.YearQuarter % 10 

Market_returns = pd.read_excel(path, sheet_name='Market returns', skiprows=5)
Market_returns = Market_returns.set_index(['YearQuarter'])  

Market_returns_1p = Market_returns.shift(-1)
Market_returns_1 = Market_returns.shift(1)
Market_returns_2 = Market_returns.shift(2)
Market_returns_3 = Market_returns.shift(3)
Market_returns_4 = Market_returns.shift(4)
#Market_returns['Quarter'] = Liquidity.YearQuarter % 10 


#names_of_variables = [Issues, Issues_1]

#Issues = Issues.drop(columns=['Quarter']).set_index(['YearQuarter'])
#Issues = pd.DataFrame(Issues.stack(dropna = False ), columns=['Issues'])
#Issues.index = Issues.index.set_names(['YearQuarter', 'Country'])
#Issues

#Liquidity = Liquidity.drop(columns=['Quarter']).set_index(['YearQuarter'])
#Liquidity = pd.DataFrame(Liquidity.stack(dropna = False ), columns=['Liquidity'])
#Liquidity.index = Liquidity.index.set_names(['YearQuarter', 'Country'])
#Liquidity

#Market_returns = Market_returns.drop(columns=['Quarter']).set_index(['YearQuarter'])
#Market_returns = pd.DataFrame(Market_returns.stack(dropna = False ), columns=['Market_returns'])
#Market_returns.index = Market_returns.index.set_names(['YearQuarter', 'Country'])
#Market_returns




names_of_variables = [Issues, 
                      Liquidity_2p, Liquidity_1p, Liquidity,
                      Liquidity_1, Liquidity_2, Liquidity_3, Liquidity_4,
                      Liquidity_5, Liquidity_6,
                      Market_returns_1p, Market_returns, Market_returns_1,
                      Market_returns_2, Market_returns_3, Market_returns_4,
                      Issues_1, Issues_2, Issues_3]

list_of_variables = ['Issues', 
                      'Liquidity_2p', 'Liquidity_1p', 'Liquidity',
                      'Liquidity_1', 'Liquidity_2', 'Liquidity_3', 'Liquidity_4',
                      'Liquidity_5', 'Liquidity_6',
                      'Market_returns_1p', 'Market_returns', 'Market_returns_1',
                      'Market_returns_2', 'Market_returns_3', 'Market_returns_4',
                      'Issues_1', 'Issues_2', 'Issues_3']

i = 0
for data in names_of_variables:
  #data = data.set_index(['YearQuarter'])
  data  = pd.DataFrame(data.stack(dropna = False ), columns=[list_of_variables[i]])
  data.index = data.index.set_names(['YearQuarter', 'Country'])
  if i == 0: new_name_of_variables = pd.DataFrame(index = data.index, columns= list_of_variables)
  new_name_of_variables[list_of_variables[i]] = data
  i += 1

#new_name_of_variables = new_name_of_variables.diff()

panel_data = pd.concat([new_name_of_variables,
                        pd.get_dummies(pd.DataFrame(new_name_of_variables.index.get_level_values(0) % 10, 
                                                index = new_name_of_variables.index)['YearQuarter'])], axis = 1)



panel_data                                                

Issues  Liquidity_2p  Liquidity_1p  ...  2  3  4
YearQuarter Country                                          ...         
19901       ARG             NaN           NaN           NaN  ...  0  0  0
            AUS             NaN           NaN           NaN  ...  0  0  0
            AUT             NaN           NaN           NaN  ...  0  0  0
            BEL             NaN           NaN           NaN  ...  0  0  0
            BRA             NaN           NaN           NaN  ...  0  0  0
...                         ...           ...           ...  ... .. .. ..
20144       SWE        1.498864           NaN           NaN  ...  0  0  1
            THA        1.521491           NaN           NaN  ...  0  0  1
            USANASDAQ  1.481866           NaN           NaN  ...  0  0  1
            USANYSE    0.079183           NaN           NaN  ...  0  0  1
            ZAF        2.606112           NaN           NaN  ...  0  0  1

[4000 rows x 23 columns]

In [ ]:
panel_data = panel_data.reset_index().set_index(['Country', 'YearQuarter'])
panel_data

model = PanelOLS(panel_data.dropna().Issues,
                 panel_data.dropna().drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5', 'Liquidity_6', 'Issues_2', 'Issues_3']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.4747
Estimator:                   PanelOLS   R-squared (Between):             -0.3087
No. Observations:                3028   R-squared (Within):               0.4748
Date:                Sun, Jul 25 2021   R-squared (Overall):              0.4747
Time:                        12:19:28   Log-likelihood                   -4943.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      170.06
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.700   Distribution:                 F(16,3011)
Min Obs:                       0.0000                                           
Max Obs:                       89.000   F-statistic (robust):             169.09
                            

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues, panel_data.fillna(0).drop(columns =['Issues','Liquidity_2p','Liquidity_5','Liquidity_6','Issues_2','Issues_3']))

res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.4558
Estimator:                   PanelOLS   R-squared (Between):             -0.7399
No. Observations:                4000   R-squared (Within):               0.4558
Date:                Sun, Jul 25 2021   R-squared (Overall):              0.4558
Time:                        12:19:31   Log-likelihood                   -6209.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      208.49
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(16,3983)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             208.43
                            

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues,
                 panel_data.fillna(0).drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5', 'Liquidity_6', 'Issues_2', 'Issues_3']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.4558
Estimator:                   PanelOLS   R-squared (Between):             -0.7399
No. Observations:                4000   R-squared (Within):               0.4558
Date:                Sun, Jul 25 2021   R-squared (Overall):              0.4558
Time:                        12:19:36   Log-likelihood                   -6209.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      208.49
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(16,3983)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             208.43
                            

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues,
                 panel_data.fillna(0).drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5', 'Liquidity_6', 'Issues_2', 'Issues_3']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.4558
Estimator:                   PanelOLS   R-squared (Between):             -0.7399
No. Observations:                4000   R-squared (Within):               0.4558
Date:                Sun, Jul 25 2021   R-squared (Overall):              0.4558
Time:                        12:19:39   Log-likelihood                   -6209.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      208.49
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(16,3983)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             208.43
                            

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues,
                 panel_data.fillna(0).drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5', 'Liquidity_6', 'Issues_2', 'Issues_3']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.4558
Estimator:                   PanelOLS   R-squared (Between):             -0.7399
No. Observations:                4000   R-squared (Within):               0.4558
Date:                Sun, Jul 25 2021   R-squared (Overall):              0.4558
Time:                        12:20:30   Log-likelihood                   -6209.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      208.49
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(16,3983)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             208.43
                            

In [ ]:
model = PanelOLS(panel_data.dropna().Issues,
                 panel_data.dropna().drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5', 'Liquidity_6']))
res = model.fit()

print(res.summary)

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues, panel_data.fillna(0).drop(columns = ['Issues','Liquidity_2p','Liquidity_5','Liquidity_6']))
res = model.fit()




model = PanelOLS(panel_data.fillna(0).Issues,
                 panel_data.fillna(0).drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5', 'Liquidity_6']))
res = model.fit()

print(res.summary)